# Homework 3

TODO:
* check input and returns in functions
* check that what i wrote is accurate

Leonardo Masci, Jeremy Sapienza, Antonio Zappia

The aim of this homework is to create a search engine that returns some book suggestions, given an input from the user in the form of words. To achieve this, we first scraped the website Goodreads.com, in its "best books ever" section, to get a dataset of 30,000 books. Once we had our dataset, we created increasingly better search algorithms to get to a satisfying result. 

Finally, in the last exercise we faced a recursive algorithm and its running time.

## 1) Data Collection

Since no dataset was provided for this homework, we had to create our own dataset by scraping the given website, which was done in the following steps.

### 1.1 Get the list of books

Firstly, we had to get the list of all the books we wanted to add to our dataset. This included all the books found in the first 300 pages of the "best books ever" section.
Therefore, we created a function that scrapes a certain number of pages from an initial url, to extract the url of each of them. The initial url and the following number of pages are given as input, while the resulting urls are stored in a txt file called "urlpages.txt" .

In [23]:
functions.get_urls("https://www.goodreads.com/list/show/1.Best_Books_Ever?page=", 300)

### 1.2 Crawl books

Once we had our txt file containing all the urls, we created a second function that downloads the html of each given url. These htmls are collected in different folders, depending on the corresponding page. Moreover, we added a try and except to be sure that all requests that we could lose will be re-scrape in a second moment.

In [24]:
functions.crawl_urls(300,"urlpages.txt")

### 1.3 Parse downloaded pages

Finally, after completing this step, we went on to actually parse the given html articles. We will obtain for each html article its article_i.tsv.

In [ ]:
functions.get_scraping("html_pages")

### Creating and concatening the final dataset for our purposes

Since running the above mentioned functions took a long time, the three of us split the work among us. So in the end we run another function that returned our distinct outputs as tsv files and combined those three to create the final dataset we will be working with.

In [ ]:
functions.finaldataset_tsv(html_pages,1)

## 2) Search Engine

Now that we have our dataset, we can start working on the search engines we are interested in. We will start with a very simple one and work our way towards a more complex one.

In [3]:
#load the dataset
df = pd.read_csv("finaloutput.tsv", sep=",")
df = df.reset_index(drop=True) # reset index

### Cleaning data and creating the index

Before we can procede with the search engines, we need to clean the dataset we have obtained and create a vocabulary. 

The clean data is needed to apply the search engine, as the input words may not match the ones in our file if processed raw. For this reason, we remove stopwords (such as 'a', 'the', etc.) and punctuation, which do not constitute valuable information about the text, and then we apply stemming. This way, we are left with only the root of words, which will make matching in the coming functions that much easier. These are all steps of the method Bag of Words, which has the ultimate goal of creating a vocabulary of all the unique words found in the dataframe.

That is why we defined a copy of the original dataset, as the clean version of the dataset itself. This way we can create the needed vocabulary, which will be based on the 'Plot' column.

In [79]:
df_copy = functions.cleaningDataset(df.copy()) # useful to obtain the vocabulary for each token

After cleaning the dataset with apposite bag of words method, we create the vocabulary. The vocabulary is needed to create the inverted list as a dictionary and at the same time to map each word with its 'term_id_i'. We create this to avoid calculating the data structure every time.

In [ ]:
vocabulary = functions.createVocabulary(df_copy)

To avoid having to calculate it over and over, we save the vocabulary as csv file.

In [ ]:
functions.create_csv(vocabulary)

### Execute the query

We are now ready to receive the user's input, which is then processed the same way as the dataset was so that the matching will be possible.

In [4]:
query = input("Insert your query string: ")

Insert your query string: Survival games


Here we obtain the clean query, which can be matched with the vocabulary tokens.

In [13]:
cleanQString = functions.cleanQuery(query)
cleanQString

['surviv', 'game']

# Why is this separated from the rest of vocabulary??

In [215]:
vocabulary = functions.open_vocabulary()

,Word,Term_id,Document_List
0,could,term_id_1,"['document_0', 'document_26', 'document_38']"
1,surviv,term_id_2,"['document_0', 'document_26', 'document_40', '..."
2,wild,term_id_3,"['document_0', 'document_8', 'document_19', 'd..."
3,everi,term_id_4,"['document_0', 'document_3', 'document_5', 'do..."
4,one,term_id_5,"['document_0', 'document_9', 'document_14', 'd..."
...,...,...,...
1988,manheim,term_id_1989,['document_49']
1989,1996,term_id_1990,['document_49']
1990,access,term_id_1991,['document_49']
1991,poem,term_id_1992,['document_49']


Finally, we create the inverted list based on that vocabulary. What is it? Why is it needed?

In [ ]:
functions.inverted_list(vocabulary)

### 1st Search Engine

Finally, we can create the new search engine by using the clean query and the inverted list we have found in the previous steps.

In [ ]:
functions.search_engine1(cleanQString,'inv_lst.csv')

<font color="red"> YES, COMMENT IT... shows that is like the professor's request! </font>
non c'è un cavolo da commentare se ancora non è runnata la cosa ahah

### 2.2 Conjunctive query & Ranking score

Before defining the new search engine, we have to update our inverted list according to professors request, we want to have the pair list, for each term that we have into our vocabulary, the document where the term is present and the relative tf_idf score.

We get the term frequency by simply counting the amount of times each term of the query appears in each document. Next we calculate the idf (inverse document frequency) as a way to give priority to those terms which are found less commonly in the document set. As the name suggests, this is an _inverse frequency_. Finally, we combine the two in the tf_idf, which is nothing more than their product.

In [ ]:
functions.new_inv_lst('inv_lst.csv',vocabulary)

### 2nd Search Engine

Again, we get the user's input and we clean it up.

In [289]:
query = input("Insert your query string: ")

Insert your query string: could wild


In [290]:
cleanQString = functions.cleanQuery(query)
cleanQString

['could', 'wild']

We can now run the second search engine, which is an improved version of the previous one as it takes into consideration not only the number of words in the query that are found in the plots, but also how often those words are mentioned. This makes the search engine more accurate in identifying books that are relevant given a certain query, which is in line with our final goals.

In [311]:
final_df = functions.search_engine2()

Finally, we illustrate the final dataset obtained with our second search engine. We chose to only see the top 5 suggestions, which are sorted by similarity score in the following table.

In [312]:
final_df

,bookTitle,Plot,Url,Similarity
0,Where the Wild Things Are,"Max, a wild and naughty boy, is sent to bed wi...",https://www.goodreads.com/book/show/19543.Wher...,0.11
1,The Picture of Dorian Gray,"Written in his distinctively dazzling manner, ...",https://www.goodreads.com/book/show/5297.The_P...,0.05
2,A Wrinkle in Time,It was a dark and stormy night.Out of this wil...,https://www.goodreads.com/book/show/33574273-a...,0.03
3,The Hunger Games,"Could you survive on your own in the wild, wit...",https://www.goodreads.com/book/show/2767052-th...,0.03
4,The Lightning Thief,Alternate cover for this ISBN can be found her...,https://www.goodreads.com/book/show/28187.The_...,0.01


## 3) Define a new score

We now want to improve again on our search engine, this time by taking into account multiple variables. 
* similarity score also in book title and author (year it was published, series, characters and setting)
* we could define a "popularity" score that takes into account the average rating and the number of reviews
* we then could combine the two to create the final score and sorting order